In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import tensorflow as tf
import numpy as np
from glob import glob
from itertools import cycle

mel_stat = np.load('../speech-bahasa/osman-stats/stats.npy')

mels = glob('../speech-bahasa/output-osman/mels/*.npy')
file_cycle = cycle(mels)
f = next(file_cycle)

In [3]:
def norm_mean_std(x, mean, std):
    zero_idxs = np.where(x == 0.0)[0]
    x = (x - mean) / std
    x[zero_idxs] = 0.0
    return x

In [4]:
path = 'melgan-osman'
ckpt_path = tf.train.latest_checkpoint(path)
ckpt_path

'melgan-osman/model.ckpt-510000'

In [5]:
def generate(batch_max_steps = 8192, hop_size = 256):
    while True:
        f = next(file_cycle)
        mel = np.load(f)
        mel = norm_mean_std(mel, mel_stat[0], mel_stat[1])
        audio = np.load(f.replace('mels', 'audios'))

        yield {'mel': mel, 'audio': audio}

In [6]:
dataset = tf.data.Dataset.from_generator(
    generate,
    {'mel': tf.float32, 'audio': tf.float32},
    output_shapes = {
        'mel': tf.TensorShape([None, 80]),
        'audio': tf.TensorShape([None]),
    },
)
features = dataset.make_one_shot_iterator().get_next()
features

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


{'mel': <tf.Tensor 'IteratorGetNext:1' shape=(?, 80) dtype=float32>,
 'audio': <tf.Tensor 'IteratorGetNext:0' shape=(?,) dtype=float32>}

In [7]:
import malaya_speech
import malaya_speech.train
from malaya_speech.train.model import melgan
import malaya_speech.config

melgan_config = malaya_speech.config.melgan_config
generator = melgan.Generator(
    melgan.GeneratorConfig(**melgan_config['melgan_generator_params']),
    name = 'melgan-generator',
)

In [8]:
y_hat = generator([features['mel']], training = False)
y_hat

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


<tf.Tensor 'melgan-generator/sequential/activation/Tanh:0' shape=(1, ?, 1) dtype=float32>

In [9]:
mel = tf.placeholder(tf.float32, [None, None, 80])
y_hat_ = generator(mel, training = False)
y_hat_

<tf.Tensor 'melgan-generator_1/sequential/activation/Tanh:0' shape=(?, ?, 1) dtype=float32>

In [10]:
y_hat_ = tf.identity(y_hat_, name = 'logits')

In [11]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [12]:
var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
saver = tf.train.Saver(var_list = var_list)
saver.restore(sess, ckpt_path)

INFO:tensorflow:Restoring parameters from melgan-osman/model.ckpt-510000


In [13]:
import IPython.display as ipd

In [14]:
f, y_ = sess.run([features, y_hat])

In [15]:
ipd.Audio(f['audio'], rate = 22050)

In [16]:
ipd.Audio(y_[0,:,0], rate = 22050)

In [17]:
import pickle

with open('a.pkl', 'rb') as fopen:
    data = pickle.load(fopen)

In [18]:
len(data[0])

1758

In [19]:
y_ = sess.run(y_hat_, feed_dict = {mel: [data[0]]})
ipd.Audio(y_[0,:,0], rate = 22050)

In [20]:
saver = tf.train.Saver()
saver.save(sess, 'melgan-osman-output/model.ckpt')

'melgan-osman-output/model.ckpt'

In [21]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'gather' in n.op.lower()
        or 'Placeholder' in n.name
        or 'logits' in n.name)
        and 'adam' not in n.name
        and 'global_step' not in n.name
        and 'Assign' not in n.name
        and 'ReadVariableOp' not in n.name
        and 'Gather' not in n.name
    ]
)
strings.split(',')

['Placeholder', 'logits']

In [22]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [23]:
freeze_graph('melgan-osman-output', strings)

INFO:tensorflow:Restoring parameters from melgan-osman-output/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 84 variables.
INFO:tensorflow:Converted 84 variables to const ops.
1038 ops in the final graph.


In [24]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
                
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
        
    return graph

In [25]:
g = load_graph('melgan-osman-output/frozen_model.pb')

In [26]:
test_sess = tf.InteractiveSession(graph = g)

/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [27]:
X = g.get_tensor_by_name(f'import/Placeholder:0')
logits = g.get_tensor_by_name(f'import/logits:0')

In [28]:
# y_ = test_sess.run(logits, feed_dict = {X: [data[1]]})
# ipd.Audio(y_[0,:,0], rate = 22050)

In [29]:
from tensorflow.tools.graph_transforms import TransformGraph

In [30]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-1024, fallback_max=1024)',
             'strip_unused_nodes',
             'sort_by_execution_order']

In [31]:
pb = 'melgan-osman-output/frozen_model.pb'

In [32]:
input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

transformed_graph_def = TransformGraph(input_graph_def, 
                                           ['Placeholder'],
                                           ['logits'], transforms)
    
with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

Instructions for updating:
Use tf.gfile.GFile.


In [33]:
import os
b2_application_key_id = os.environ['b2_application_key_id']
b2_application_key = os.environ['b2_application_key']

In [34]:
from b2sdk.v1 import *
info = InMemoryAccountInfo()
b2_api = B2Api(info)
application_key_id = b2_application_key_id
application_key = b2_application_key
b2_api.authorize_account("production", application_key_id, application_key)
file_info = {'how': 'good-file'}
b2_bucket = b2_api.get_bucket_by_name('malaya-speech-model')

In [35]:
file = 'melgan-osman-output/frozen_model.pb'
outPutname = 'vocoder-melgan/osman/model.pb'
b2_bucket.upload_local_file(
    local_file=file,
    file_name=outPutname,
    file_infos=file_info,
)

In [36]:
file = 'melgan-osman-output/frozen_model.pb.quantized'
outPutname = 'vocoder-melgan/osman-quantized/model.pb'
b2_bucket.upload_local_file(
    local_file=file,
    file_name=outPutname,
    file_infos=file_info,
)

In [37]:
!cp melgan-osman/*.tfevents.* melgan-osman-output
!tar -cf melgan-osman-output.tar melgan-osman-output

In [38]:
from malaya_boilerplate.huggingface import upload_dict

In [39]:
files_mapping = {'melgan-osman-output/frozen_model.pb': 'model.pb'}
upload_dict(model = 'vocoder-melgan-osman', files_mapping = files_mapping)

409 Client Error: Conflict for url: https://huggingface.co/api/repos/create - You already created this model repo


In [40]:
files_mapping = {'melgan-osman-output/frozen_model.pb.quantized': 'model.pb'}
upload_dict(model = 'vocoder-melgan-osman-quantized', files_mapping = files_mapping)

409 Client Error: Conflict for url: https://huggingface.co/api/repos/create - You already created this model repo


In [41]:
files_mapping = {'melgan-osman-output.tar': 'melgan-osman-output.tar'}
upload_dict(model = 'pretrained-vocoder', files_mapping = files_mapping)

409 Client Error: Conflict for url: https://huggingface.co/api/repos/create - You already created this model repo
